In [1]:
library(readxl)


Warning message:
"程辑包'readxl'是用R版本4.3.3 来建造的"


In [ ]:
url <- "https://raw.githubusercontent.com/Speculatorrr/ECON326/d0e554d0459664e17c4489ca8c08f9e1f0e2406c/Student%20Performance%20Survey%20-%20March%202025.xlsx"
temp_file <- tempfile(fileext = ".xlsx")
download.file(url, temp_file, mode = "wb")
df <- read_excel(temp_file)
head(df)

1. What is your current cumulative GPA (on a percentage scale)? Example: 75%,2. What is your current GPA on a 4.0 scale (if known)? Example: 3.51,3. How many hours per week do you study for this course? Example: 9,4. How many total hours per week do you study across all your courses? Example: 21,5. How many hours do you spend preparing for exams (excluding regular coursework)? Example: 8,6. How many courses do you take on average per term? Example: 4,7. Do you prefer courses that focus more on practical applications or theoretical concepts?,8. How many hours per week do you use ChatGPT or other LLM or AI tools for studying or assignments? Example: 12,"9. How do you feel AI tools like ChatGPT impact your learning? (5 = Strongly helps, 1 = Strongly harms)",10. How much time per week do you spend on reviewing lecture slides and course materials? Example: 6,⋯,"28. How long is your commute to campus (one way, in minutes)? Example: 25. If you live on campus, just write the walk time on average in minutes to your classes.",29. What is your primary mode of transportation to campus?,"30. Do you prefer taking online courses, in-person courses, or a hybrid of both?","31. How often do you think about your future employment or career prospects? (5 = Constantly, 1 = Never)","32. Do you plan to pursue further education (graduate school, professional certification, etc.)?","33. How confident are you in securing a job related to your field of study after graduation? (5 = Very confident, 1 = Not confident at all)",34. Which career path are you most likely to pursue after graduation? (Select the option that best matches your current career goal),35. Which faculty are you in? (Select the faculty that best matches your program of study). Faculty of Arts is very vast so please google or use AI.,36. What is your ethnicity?,"37. Which year are you in at your undergraduate program at UBC? (1 = First Year, 5 = Fifth Year or beyond)"
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
86,3.76,8,30,30,4.8,More practical applications,1,5,2,⋯,25,Public transit,Hybrid (mix of both),3,Yes,3,"Finance, Banking, or Investment",Faculty of Arts,East Asian,3
78.6%,3.38,1,5,4,5,More practical applications,2,5,1,⋯,20,Walking,Online courses,5,Unsure,1,"Finance, Banking, or Investment",Faculty of Commerce and Business Administration (Sauder School of Business),East Asian,3
63,2.4,9,21,8,4,More practical applications,12,5,6,⋯,25,Public transit,Online courses,5,Yes,5,"Finance, Banking, or Investment",Faculty of Arts,East Asian,1
73%,unknown,4,12,20,4.5,More practical applications,almost 0,1,2,⋯,20,Walking,In-person courses,5,Unsure,5,Marketing or Communications,Faculty of Arts,East Asian,3
74.2,3.2,1,10-12,4-5 a day?,4,More theoretical concepts,2-3 hours,5,3-4 hours,⋯,10,Walking,In-person courses,4,Unsure,5,Date Science or AI,Faculty of Arts,East Asian,3
90.1%,4,5,30,20,4.3,More practical applications,10,5,7,⋯,20,Walking,In-person courses,3,No,4,Entrepreneurship or Startups,Faculty of Arts,East Asian,3


In [5]:
colnames(df)

[1] "1. What is your current cumulative GPA (on a percentage scale)? Example: 75%"                                                                                                    
 [2] "2. What is your current GPA on a 4.0 scale (if known)? Example: 3.51"                                                                                                            
 [3] "3. How many hours per week do you study for this course? Example: 9"                                                                                                             
 [4] "4. How many total hours per week do you study across all your courses? Example: 21"                                                                                              
 [5] "5. How many hours do you spend preparing for exams (excluding regular coursework)? Example: 8"                                                                                   
 [6] "6. How many courses do you take on average per term? Example: 4"                                                                                                                 
 [7] "7. Do you prefer courses that focus more on practical applications or theoretical concepts?"                                                                                     
 [8] "8.  How many hours per week do you use ChatGPT or other LLM or AI tools for studying or assignments? Example: 12"                                                                
 [9] "9. How do you feel AI tools like ChatGPT impact your learning? (5 = Strongly helps, 1 = Strongly harms)"                                                                         
[10] "10. How much time per week do you spend on reviewing lecture slides and course materials? Example: 6"                                                                            
[11] "11. How much time per week do you spend on assignments, problem sets, and practice exercises? Example: 5"                                                                        
[12] "12. How many hours per week do you spend on sports or fitness activities? Example: 4"                                                                                            
[13] "13. How many hours per week do you spend on student clubs or societies? Example: 5"                                                                                              
[14] "14. How many hours per week do you spend on volunteering or community service? Example: 3"                                                                                       
[15] "15. How many hours per week do you spend socializing with friends? Example: 6"                                                                                                   
[16] "16. How many hours per week do you spend socializing with family (online and in-person together)? Example: 8"                                                                    
[17] "17. Are you an international, domestic, or exchange student? If you are an exchange student, just choose \"Exchange\" below."                                                    
[18] "18. What is your average tuition fee per course? Example: 3200"                                                                                                                  
[19] "19. Do you receive any scholarships or financial aid?"                                                                                                                           
[20] "20. If you receive scholarships or financial aid, what percentage of your tuition does it cover? Example: 50%. If you do not get scholarship or financial aid, just enter \"0\"."
[21] "21. Do you currently have a job while studying?"                                                                                                                                 
[22] "22. How many hours per week do you work? Example: 12. Put \"0\" if you are not working."                                           

In [6]:
selected_df <- df[, c(1,2,18,20,23)]